In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, TimestampType
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace, col, trim
import re
from functools import reduce
from pyspark.sql import DataFrame

In [0]:
print(">>> Criando schema gold no catálogo de dados, caso ainda não exista...", end=" ")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS gold_layer")
print("OK!")

In [0]:
%run /Workspace/Users/douglasfmedeiros@outlook.com/desafio_neurotech/functions/gold_functions

In [0]:
# Lista todas as tabelas existentes na camada silver
tabelas = spark.catalog.listTables("silver_layer");
tabelas = [t for t in tabelas if t.name != "_sqldf"]

In [0]:
for tabela in tabelas:
    nome_variavel = tabela.name.replace('iptu', 'dados')
    globals()[nome_variavel] = spark.read.table(f'silver_layer.{tabela.name}')

dfs = [eval(t.name.replace('iptu', 'dados')) for t in tabelas]
nomes = [tabela.name for tabela in tabelas]

In [0]:
df_final = dfs[0]
for df in dfs[1:]:
    df_final = df_final.unionByName(df, allowMissingColumns=True)

In [0]:
ingestao_dados(df_final, 'dados_iptu')

In [0]:
print(f'/////////////////////////////////////////// INGESTÃO DA CAMADA GOLD FINALIZADA ///////////////////////////////////////////')